# Get Returns 

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


dates = pd.read_csv('dates/dates_updated.csv')
dates = dates.drop(index=[59, 67, 161, 163, 164, 168]).reset_index(drop=True)
dates_new = dates
dates_new.to_csv('dates/dates_new.csv')


In [2]:
# Load Dates for Statements & Intermeetings
dates_statements = dates_new
dates_statements['Statement Date'] = pd.to_datetime(dates_statements['Statement Date'], errors='coerce')
dates_statements['document_type'] = 'statement'

dates_intermeetings = dates_new
dates_intermeetings['Intermeeting Date'] = pd.to_datetime(dates_intermeetings['Intermeeting Date'], errors='coerce')
dates_intermeetings['document_type'] = 'intermeeting'

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_16922/4073433599.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates_statements['Statement Date'] = pd.to_datetime(dates_statements['Statement Date'], errors='coerce')
/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_16922/4073433599.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates_intermeetings['Intermeeting Date'] = pd.to_datetime(dates_intermeetings['Intermeeting Date'], errors='coerce')


In [3]:
tickers = [
    '^GSPC', '^IXIC', '^DJI', '^RUT', '^W5000',
    'XLF', 'XLRE', 'XLU', 'XLY', 'XLP',
    'XLE', 'XLV', 'XLI', 'XLB', 'XLK', 'XLC',
    '^IRX', '^TNX'
]


In [4]:
dates_new

,Statement Date,Intermeeting Date,Chair,document_type
0,2000-02-02,2000-03-23,Alan Greenspan,intermeeting
1,2000-03-21,2000-05-18,Alan Greenspan,intermeeting
2,2000-05-16,2000-06-29,Alan Greenspan,intermeeting
3,2000-06-28,2000-08-24,Alan Greenspan,intermeeting
4,2000-08-22,2000-10-05,Alan Greenspan,intermeeting
...,...,...,...,...
195,2024-09-18,2024-10-09,Jerome Powell,intermeeting
196,2024-11-07,2024-11-26,Jerome Powell,intermeeting
197,2024-12-18,2025-01-08,Jerome Powell,intermeeting
198,2025-01-29,2025-02-19,Jerome Powell,intermeeting


In [5]:

dates['Statement Date'] = pd.to_datetime(dates['Statement Date'])
dates['document_type'] = 'statement' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Statement Date'].min() - pd.Timedelta(days=15)
end_date = dates['Statement Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Statement Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

statements_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
statements_df = statements_df[column_order]

statements_df

statements_df.to_csv('raw_data/statement_prices.csv', index=False)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
# First, ensure the announcement_date column is datetime (if it isn't already)
statements_df['announcement_date'] = pd.to_datetime(statements_df['announcement_date'])

# Now create a meeting_id based on unique announcement_date
statements_df['meeting_id'] = statements_df['announcement_date'].factorize()[0] + 1


In [7]:
statements_df

,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,T-9,...,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15,meeting_id
0,2000-02-02,^GSPC,statement,<NA>,0.000522,-0.007095,-0.002912,<NA>,<NA>,-0.027634,...,-0.020815,0.003627,-0.020969,<NA>,<NA>,0.002033,0.008713,-0.010256,0.000425,1
1,2000-02-02,^IXIC,statement,<NA>,0.004958,0.009207,0.010954,<NA>,<NA>,-0.032894,...,-0.014514,0.02805,-0.020104,<NA>,<NA>,0.005255,0.000502,0.001556,0.027389,1
2,2000-02-02,^DJI,statement,<NA>,-0.006173,-0.012016,-0.008773,<NA>,<NA>,-0.021645,...,-0.023585,-0.00519,-0.020521,<NA>,<NA>,0.009077,0.018845,-0.014618,-0.004435,1
3,2000-02-02,^RUT,statement,<NA>,0.012776,0.013961,0.012631,<NA>,<NA>,-0.020583,...,-0.002772,0.011586,-0.009424,<NA>,<NA>,0.005288,0.000556,0.01392,0.019461,1
4,2000-02-02,^W5000,statement,<NA>,0.003503,-0.002435,0.001411,<NA>,<NA>,-0.026098,...,-0.018755,0.005698,-0.018061,<NA>,<NA>,0.002445,0.006289,-0.005964,0.004514,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2025-03-19,XLB,statement,-0.013566,0.026107,-0.005225,0.00274,<NA>,<NA>,-0.020838,...,0.001849,-0.000461,-0.018352,<NA>,<NA>,0.010935,0.003722,0.008227,<NA>,200
3596,2025-03-19,XLK,statement,-0.000777,0.01405,-0.027757,0.014483,<NA>,<NA>,-0.042509,...,-0.022304,-0.009228,-0.024254,<NA>,<NA>,0.000484,0.007313,0.007404,<NA>,200
3597,2025-03-19,XLC,statement,-0.007922,0.012877,-0.017936,0.006422,<NA>,<NA>,-0.024928,...,-0.008505,-0.00444,-0.029498,<NA>,<NA>,0.007416,0.000622,0.003523,<NA>,200
3598,2025-03-19,^IRX,statement,0.000477,0.002384,-0.001902,0.0,<NA>,<NA>,-0.002859,...,0.001913,-0.000477,0.0,<NA>,<NA>,0.001194,0.001669,0.000714,<NA>,200


In [8]:
dates['Intermeeting Date'] = pd.to_datetime(dates['Intermeeting Date'])
dates['document_type'] = 'intermeeting' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Intermeeting Date'].min() - pd.Timedelta(days=15)
end_date = dates['Intermeeting Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Intermeeting Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

intermeeting_df = pd.DataFrame(rows)


column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
intermeeting_df = intermeeting_df[column_order]

intermeeting_df

intermeeting_df.to_csv('raw_data/intermeeting_prices.csv', index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
# First, ensure the announcement_date column is datetime (if it isn't already)
intermeeting_df['announcement_date'] = pd.to_datetime(intermeeting_df['announcement_date'])

# Now create a meeting_id based on unique announcement_date
intermeeting_df['meeting_id'] = intermeeting_df['announcement_date'].factorize()[0] + 1


In [10]:
intermeeting_df

,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,T-9,...,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15,meeting_id
0,2000-03-23,^GSPC,intermeeting,<NA>,0.025602,-0.004723,<NA>,<NA>,-0.008207,-0.017685,...,-0.013656,0.007164,<NA>,<NA>,0.004931,-0.007464,-0.004924,0.009392,0.009998,1
1,2000-03-23,^IXIC,intermeeting,<NA>,0.030548,0.000349,<NA>,<NA>,-0.028004,-0.04088,...,-0.040214,0.025783,<NA>,<NA>,-0.076353,-0.017707,0.0049,0.023587,0.041919,1
2,2000-03-23,^DJI,intermeeting,<NA>,0.015644,-0.008182,<NA>,<NA>,0.001844,-0.013661,...,-0.003491,-0.005312,<NA>,<NA>,0.027469,-0.005087,-0.011726,0.007282,-0.000251,1
3,2000-03-23,^RUT,intermeeting,<NA>,0.01912,-0.003696,<NA>,<NA>,-0.02264,-0.029061,...,-0.02105,0.014147,<NA>,<NA>,-0.042757,-0.019223,0.023552,0.027913,0.0197,1
4,2000-03-23,^W5000,intermeeting,<NA>,0.024762,-0.00388,<NA>,<NA>,-0.01357,-0.021922,...,-0.01743,0.011479,<NA>,<NA>,-0.012209,-0.011746,-0.000518,0.014204,0.013746,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2025-04-09,XLB,intermeeting,-0.000231,0.001849,-0.000461,-0.018352,<NA>,<NA>,0.010935,...,-0.007794,0.006858,<NA>,<NA>,<NA>,-0.015232,0.02339,0.001229,<NA>,200
3596,2025-04-09,XLK,intermeeting,0.003308,-0.022304,-0.009228,-0.024254,<NA>,<NA>,0.000484,...,-0.034745,-0.005415,<NA>,<NA>,<NA>,-0.025822,0.024377,0.028993,<NA>,200
3597,2025-04-09,XLC,intermeeting,0.012666,-0.008505,-0.00444,-0.029498,<NA>,<NA>,0.007416,...,-0.026924,0.006471,<NA>,<NA>,<NA>,-0.022503,0.026083,0.016026,<NA>,200
3598,2025-04-09,^IRX,intermeeting,0.0,0.001913,-0.000477,0.0,<NA>,<NA>,0.001194,...,0.001905,-0.000713,<NA>,<NA>,<NA>,0.001189,0.000713,-0.001187,<NA>,200


In [11]:
intermeeting_df.to_csv('raw_data/intermeeting_prices.csv', index=False)

In [ ]:
combined_df = pd.concat([statements_df, intermeeting_df], ignore_index=True)
combined_df = combined_df.dropna(subset=['announcement_date'])
combined_df = combined_df.sort_values(by='announcement_date').reset_index(drop=True)
combined_df['meeting_id'] = combined_df['announcement_date'].rank(method='dense').astype(int)
combined_df = combined_df[['meeting_id'] + [col for col in combined_df.columns if col != 'meeting_id']]
combined_df.to_csv('raw_data/combined_dates.csv', index = False)

In [ ]:
# Load Combined Returns Data
combined_returns = pd.read_csv('raw_data/combined_dates.csv')
print(combined_returns.columns.tolist())

# Define T-10 to T+10 columns
t_columns = [f'T{t}' if t < 0 else f'T+{t}' if t > 0 else 'T0' for t in range(-10, 11)]

# Build dictionary by ticker
all_indices_data = {}

for ticker in combined_returns['ticker'].unique():
    ticker_data = combined_returns[combined_returns['ticker'] == ticker]
    ticker_data = ticker_data.set_index('announcement_date')  # Set date as index
    ticker_data.index = pd.to_datetime(ticker_data.index)  # Ensure datetime index
    all_indices_data[ticker] = ticker_data

print("All index data loaded.")

In [ ]:
all_indices_data['^DJI']['T+0']

In [ ]:
# Define your get_trading_window function
def get_trading_window(trading_dates, fed_date, returns_series, window=10):
    try:
        fed_idx = trading_dates.get_loc(fed_date)
    except KeyError:
        idx_array = trading_dates.get_indexer([fed_date], method='ffill')
        fed_idx = idx_array[0]
        if fed_idx == -1:
            return None

    if fed_idx < 0 or fed_idx >= len(returns_series):
        return None

    result = {}
    for t in range(-window, 0):
        if 0 <= fed_idx + t < len(returns_series):
            result[f'T{t:+}'] = returns_series.iloc[fed_idx + t]
        else:
            result[f'T{t:+}'] = pd.NA
    if 0 <= fed_idx < len(returns_series):
        result['T0'] = returns_series.iloc[fed_idx]
    else:
        result['T0'] = pd.NA
    for t in range(1, window + 1):
        if 0 <= fed_idx + t < len(returns_series):
            result[f'T{t:+}'] = returns_series.iloc[fed_idx + t]
        else:
            result[f'T{t:+}'] = pd.NA

    return result

In [ ]:
# Process Statement Prices
rows = []

for idx, row_fomc in dates_statements.iterrows():
    date = row_fomc['Statement Date']
    document_type = row_fomc['document_type']

    for ticker in all_indices_data.keys():
        ticker_data = all_indices_data[ticker]

        if date not in ticker_data.index:
            continue  # skip if no data

        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type}

        available_returns = ticker_data.loc[date]

        # Find valid pre-event returns
        pre_event_returns = []
        for t in range(-15, 0):
            col_name = f'T{t}'
            if col_name in available_returns and pd.notna(available_returns[col_name]):
                pre_event_returns.append((t, available_returns[col_name]))
            if len(pre_event_returns) == 10:
                break

        # Find valid post-event returns
        post_event_returns = []
        for t in range(1, 16):
            col_name = f'T+{t}'
            if col_name in available_returns and pd.notna(available_returns[col_name]):
                post_event_returns.append((t, available_returns[col_name]))
            if len(post_event_returns) == 10:
                break

        # Event day (T0)
        t0_value = available_returns.get('T+0', pd.NA)

        # Always create full structure: T-10 to T-1, T0, T+1 to T+10
        for i in range(-10, 0):
            if len(pre_event_returns) >= abs(i):
                row[f'T{i}'] = pre_event_returns[i + 10 - 1][1]  # -10 is idx 0
            else:
                row[f'T{i}'] = pd.NA

        row['T0'] = t0_value

        for i in range(1, 11):
            if len(post_event_returns) >= i:
                row[f'T+{i}'] = post_event_returns[i - 1][1]
            else:
                row[f'T+{i}'] = pd.NA

        rows.append(row)

statements_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t}' if t < 0 else f'T+{t}' if t > 0 else 'T0' for t in range(-10, 11)]
for col in column_order:
    if col not in statements_df.columns:
        statements_df[col] = pd.NA
statements_df = statements_df[column_order]

print("Statement prices processed.")

In [ ]:
# Process Intermeeting Prices
rows = []

for idx, row_fomc in dates_intermeetings.iterrows():
    date = row_fomc['Intermeeting Date']
    document_type = row_fomc['document_type']

    for ticker in all_indices_data.keys():
        ticker_data = all_indices_data[ticker]

        if date not in ticker_data.index:
            continue  # skip if no data

        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type}

        available_returns = ticker_data.loc[date]

        # Find valid pre-event returns
        pre_event_returns = []
        for t in range(-15, 0):
            col_name = f'T{t}'
            if col_name in available_returns.index and pd.notna(available_returns[col_name]):
                pre_event_returns.append((t, available_returns[col_name]))
            if len(pre_event_returns) == 10:
                break

        # Find valid post-event returns
        post_event_returns = []
        for t in range(1, 16):
            col_name = f'T+{t}'
            if col_name in available_returns.index and pd.notna(available_returns[col_name]):
                post_event_returns.append((t, available_returns[col_name]))
            if len(post_event_returns) == 10:
                break

        # Event day (T0)
        t0_value = available_returns.get('T+0', pd.NA)

        # Always create full structure: T-10 to T-1
        for i in range(-10, 0):
            idx = abs(i) - 1
            if idx < len(pre_event_returns):
                row[f'T{i}'] = pre_event_returns[idx][1]
            else:
                row[f'T{i}'] = pd.NA
        
        # Event day
        row['T0'] = t0_value
        
        # T+1 to T+10
        for i in range(1, 11):
            idx = i - 1
            if idx < len(post_event_returns):
                row[f'T+{i}'] = post_event_returns[idx][1]
            else:
                row[f'T+{i}'] = pd.NA


        rows.append(row)

intermeeting_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t}' if t < 0 else f'T+{t}' if t > 0 else 'T0' for t in range(-10, 11)]
for col in column_order:
    if col not in intermeeting_df.columns:
        intermeeting_df[col] = pd.NA
intermeeting_df = intermeeting_df[column_order]

print("Intermeeting prices processed.")

In [ ]:
combined_df = pd.concat([statements_df, intermeeting_df], ignore_index=True)
combined_df = combined_df.dropna(subset=['announcement_date'])
combined_df = combined_df.sort_values(by='announcement_date').reset_index(drop=True)
combined_df['meeting_id'] = combined_df['announcement_date'].rank(method='dense').astype(int)
combined_df = combined_df[['meeting_id'] + [col for col in combined_df.columns if col != 'meeting_id']]
combined_df

In [ ]:
combined_df.to_csv('raw_data/new_combined_dates.csv', index = False)

# Sentiment Analysis